# ChatBot for Personalized Book Search

### Bharath Goud Nadimpally (ID:2091905)

In [1]:
import warnings
#import SearchEngine as SE
warnings.filterwarnings("ignore")
import csv
import re
import os
import pandas as pd
import SearchEngine as SE
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag, ne_chunk
from nltk.chat.util import Chat, reflections

In [50]:
re.__version__

'2.2.1'

In [51]:
nltk.__version__

'3.7'

In [62]:
#global rec, user_id
class Chatbot:
    def __init__(self):
        self.recommendations = []
        self.user_id=[]
    def process_user_input(self,user_input):

        stop_words = set(stopwords.words('english'))
        # Tokenize the user input
        tokens = word_tokenize(user_input.lower())
        # Remove punctuation and stop words
        filtered_tokens = [word for word in tokens if  word not in stop_words]

        # Identify user intent, action, keywords, and book titles
        intent = None
        action = None
        keywords = []
        book_titles = []

        # Define keywords related to different intents and actions
        recommend_keywords = ["recommended","recommendations", "similar","similarly","recommend", "suggested", "books", "titles", "reading",
        "recommendations for", "book recommendations for",  "recommended books for","I need recommendations for", "book_id",
        "I need some recommendations for", "give me recommendations for","give me some recommendations for","wishlist",'transfer',
        "recommendations for", "book recommendations for","recommended books for", "similar books to", "books similar to",
        "books like","books related to", "books similar with","similar books for","similar book for","books similar as","copy",
        "books like the","books related with", "books related as","books related like",'bottom', 'book', 'favorite', 'list',"to_read","show"]
        add_keywords = ["add", "include", "insert", "append"]
        delete_keywords = ["delete", "remove", "discard", "eliminate"]
        select_keywords = ["select", "choose", "pick", "opt", "top"]

        for word in filtered_tokens:

            if word == "find":
                continue 
            elif word in recommend_keywords:
                keywords.append(word)
                intent = "recommend_books"
                continue
            elif word in add_keywords:
                keywords.append(word)
                action = "add"
                continue
            elif word in delete_keywords:
                keywords.append(word)
                action = "delete"
                continue
            elif word in select_keywords:
                action = "select_top"
                if "to_read" in word:
                    keywords.append(word)
                else : 
                    keywords.append(word)
                    continue

            elif (word not in recommend_keywords) and (word not in add_keywords) and (word not in delete_keywords) and (word not in select_keywords):
                if action:
                    # If an action keyword has been identified, consider the remaining words as keywords
                    keywords.append(word)
                else:
                        # If no action keyword, consider the remaining words as potential book titles
                    book_titles.append(word)   

        return book_titles, keywords



    def respond_for_input(self,inputs):

        found_keywords = []
        if (("transfer " in inputs) or ("shift" in inputs) or ("copy" in inputs))and (("to_read" in inputs) or ("wishlist" in inputs))and ("favorite" in inputs) and ("book_id" in inputs):
            found_keywords.append("transfer")
        if "transfer" in found_keywords:
            return self.input_for_transfer(inputs)
        
        if (("to_read" not in inputs) and ("wishlist" not in inputs)) and (("transfer " not in inputs) or ("shift" in inputs) or ("copy" not in inputs)) and(("recommendations" in inputs) or ("recommend" in inputs) or ("similar" in inputs) or ("find" in inputs)) and ("favorite" not in inputs):
            found_keywords.append("recommendations")
        if "recommendations" in found_keywords:
            return self.input_for_title_rec(inputs)

        if ("favorite" in inputs) or (("recommendations" in inputs) or ("recommend" in inputs) or ("similar" in inputs) or ("find" in inputs))  and (("to_read" not in inputs) and ("wishlist" not in inputs)) and (("transfer " not in inputs) or ("shift" in inputs) or ("copy" not in inputs)):
            found_keywords.append("favorite")
        if "favorite" in found_keywords: 
            return self.input_for_fav(inputs)

        if (("to_read" in inputs) or ("wishlist" in inputs)) or (("recommendations" in inputs) or ("recommend" in inputs) or ("similar" in inputs) or ("find" in inputs)) and ("favorite" not in inputs) and (("transfer " not in inputs) or ("shift" in inputs) or ("copy" not in inputs)):
            found_keywords.append("to_read")
        if "to_read" in found_keywords:
            return self.input_for_to_read(inputs)

        if not found_keywords:
            print("give command\n")

        return


    def input_for_fav(self,inputs):
        book_title, keywords = self.process_user_input(inputs)
        int_vals = []
        for item in keywords:
            try: 
                int_val = int(item)
                int_vals.append(int_val)
            except ValueError:
                pass 
        print("\nintegers in keywords : ", int_vals)
        if (("top" in keywords) or ("bottom" in keywords)) and ("favorite" in keywords) and (("recommendations" in keywords) or ("similar" in keywords) or ("related" in keywords)) and ("add" not in keywords):
            df = self.direct_input_fav(user_id)
            print("\nlist of favorite in side to find direct recommendations in fav:\n", df)
            if ("top" in keywords): 
                row = int_vals[0]
                self.recommendations= self.find_recommendations(df.iloc[row-1][2])
                print("\n",self.recommendations)
            if ("bottom" in keywords): 
                int_vals.append(1)
                row = int_vals[0]
                print("\nrow: ",row)
                self.recommendations= self.find_recommendations(df.iloc[-row][2])
                print("\n",self.recommendations)

        if len(book_title) > 0 :
            final_book_title = self.combine_words_to_title(book_title)
            if final_book_title != '':
                self.recommendations= self.find_recommendations(final_book_title)
                print("\n",self.recommendations)

        if ("add" in keywords) and (("top" in keywords) or ("bottom" in keywords)) and ("favorite" in keywords) :
            if "top" in keywords:
                row = int_vals[0]
                self.add_fav(self.recommendations.iloc[row-1][0],str(user_id))
            if "bottom" in keywords:
                int_vals.append(1)
                row = int_vals[0]
                self.add_fav(self.recommendations.iloc[-row][0], str(user_id))
            if "show" in keywords:
                df = self.direct_input_fav(user_id)
                print("\nlist of favorite :\n", df)

        if ("book_id" in keywords and "add" in keywords and "favorite" in keywords):
            self.add_fav(int_vals[0],str(user_id))
            if "show" in keywords:
                df = self.direct_input_fav(user_id)
                print("\nlist of favorite :\n", df)

        if ("book_id" in keywords and "delete" in keywords and "favorite" in keywords):
            self.delete_from_fav(str(int_vals[0]),str(user_id))
            if "show" in keywords:
                df = self.direct_input_fav(user_id)
                print("\nlist of favorite :\n", df)

        if ("delete" in keywords) and (("top" in keywords) or ("bottom" in keywords)) and ("favorite" in keywords) :
            df = self.direct_input_fav(user_id)
            if "top" in keywords:
                row = int_vals[0]
                self.delete_from_fav(df.iloc[row-1][0],str(user_id))
            elif "bottom" in keywords:
                int_vals.append(1)
                row = int_vals[0]
                print("\ndelete book_id: ", df.iloc[-row][0])
                self.delete_from_fav(df.iloc[-row][0], str(user_id))
            if "show" in keywords:
                df = self.direct_input_fav(user_id)
                print("\nlist of favorite :\n", df)
        if ("show" in keywords) and ("favorite" in keywords) and (("recommendations" not in keywords) or ("similar" not in keywords) or ("related" not in keywords)) and ("add" not in keywords) and ('delete' not in keywords):
            df = self.direct_input_fav(user_id)
            print("\nlist of favorite :\n", df)
        return

    def input_for_to_read(self,inputs):
        book_title, keywords = self.process_user_input(inputs)

        int_vals = []
        for item in keywords:
            try: 
                int_val = int(item)
                int_vals.append(int_val)
            except ValueError:
                pass 
        print("\nintegers in keywords : ", int_vals)
        if (("top" in keywords) or ("bottom" in keywords)) and (("to_read" in keywords)or ("wishlist" in keywords)) and (("recommendations" in keywords) or ("similar" in keywords) or ("related" in keywords)) and ("add" not in keywords):
            df = self.direct_input_to_read(user_id)
            print("\nlist of to_read in side to find direct recommendations in to_read:\n", df)
            if ("top" in keywords): 
                row = int_vals[0]
                self.recommendations= self.find_recommendations(df.iloc[row-1][2])
                print("\n",self.recommendations)
            if ("bottom" in keywords): 
                int_vals.append(1)
                row = int_vals[0]
                print("\nrow: ",row)
                self.recommendations= self.find_recommendations(df.iloc[-row][2])
                print("\n",self.recommendations)
        if len(book_title) > 0 :
            final_book_title = self.combine_words_to_title(book_title)
            self.recommendations= self.find_recommendations(final_book_title)
            print("\n",self.recommendations)

        if ("add" in keywords) and (("top" in keywords) or ("bottom" in keywords)) and (("to_read" in keywords)or ("wishlist" in keywords)) :
            if "top" in keywords:
                row = int_vals[0]
                self.add_to_read(self.recommendations.iloc[row-1][0],str(user_id))
            if "bottom" in keywords:
                int_vals.append(1)
                row = int_vals[0]
                self.add_to_read(self.recommendations.iloc[-row][0], str(user_id))
            if "show" in keywords:
                df = self.direct_input_to_read(user_id)
                print("\nlist of to_read :\n", df)
        if ("book_id" in keywords and "add" in keywords and (("to_read" in keywords)or ("wishlist" in keywords))):
            self.add_to_read(int_vals[0],str(user_id))
            if "show" in keywords:
                df = self.direct_input_to_read(user_id)
                print("\nlist of to_read :\n", df)

        if ("book_id" in keywords and "delete" in keywords and (("to_read" in keywords)or ("wishlist" in keywords))):
            self.delete_from_to_read(str(int_vals[0]),str(user_id))
            if "show" in keywords:
                df = self.direct_input_to_read(user_id)
                print("\nlist of to_read :\n", df)
        if ("delete" in keywords) and (("top" in keywords) or ("bottom" in keywords)) and (("to_read" in keywords)or ("wishlist" in keywords)) :
            df = self.direct_input_to_read(user_id)
            if "book_id" in keywords:
                self.delete_from_to_read(int_vals[0],str(user_id))
            else:
                if "top" in keywords:
                    row = int_vals[0]
                    self.delete_from_to_read(df.iloc[row-1][0],str(user_id))
                elif "bottom" in keywords:
                    int_vals.append(1)
                    row = int_vals[0]
                    print("\ndelete book_id: ", df.iloc[-row][0])
                    self.delete_from_to_read(df.iloc[-row][0], str(user_id))
            if "show" in keywords:
                df = self.direct_input_to_read(user_id)
                print("\nlist of to_read :\n", df)
        if ("show" in keywords) and (("to_read" in keywords) or ("wishlist" in keywords)) and (("recommendations" not in keywords) or ("similar" not in keywords) or ("related" not in keywords)) and ("add" not in keywords) and ('delete' not in keywords):
            df = self.direct_input_to_read(user_id)
            print("\nlist of to_read :\n", df)
        return

    def input_for_transfer(self,user_input):
        from_file, to_file,book_id = self.process_user_input_for_transfer(user_input)
        if from_file == "to_read":
            self.transfer_to_fav_from_to_read(book_id,user_id)
        if from_file == 'favorite':
            self.transfer_to_to_read_from_fav(book_id, user_id)
        return



    def process_user_input_for_transfer(self,user_input):
        list_to_file = {
        "to_read list": 'to_read',
        "favorite list": 'favorite'
         }
        book_id_match = re.search(r"book_id (\d+)", user_input)
        if book_id_match:  
            book_id = book_id_match.group(1)
        else:
            print("Book id not found")
            return

        from_list_match = re.search(r"from (\w+\s?list)", user_input)
        if from_list_match:
            from_list = from_list_match.group(1)
            from_file = list_to_file[from_list]
        else:
            print("Source file not found")
            return

        to_list_match = re.search(r"to (\w+\s?list)", user_input)  
        if to_list_match:   
            to_list = to_list_match.group(1)
            to_file = list_to_file[to_list]
        else:
            print("Destination file not found")
            return

        print(f"Copying book {book_id} from {from_file} to {to_file}")
        return from_file, to_file, book_id

    def find_recommendations(self,title):
        se = SE.SearchEngine()
        self.recommendations= se.query(text=title,user_id= user_id,  to_read_boosting_factor=1.5, K=10)
        print("Found recommendations for '{}'".format(title))
        return self.recommendations

    def input_for_title_rec(self,inputs):
        book_title, keywords = self.process_user_input(inputs)
        final_book_title = self.combine_words_to_title(book_title)
        self.recommendations= self.find_recommendations(final_book_title)
        print("\n",self.recommendations)

    def combine_words_to_title(self,word_list):
        keywords_to_drop = ["need", "title","i","book","name","find",'related', 'like',"give","recommendations","related","similar"]
        # Remove the selected keywords from the word list
        filtered_words = [word for word in word_list if word not in keywords_to_drop]

        # Combine the remaining words into a single string
        combined_word = ' '.join(filtered_words)
        print("\ncombined_word: that is title of the book",combined_word,"\n")
        return combined_word

    def direct_input_to_read(self,user_id):
        with open('/Users/cdmstudent/Desktop/DSC-540 Adv ML/Final Project/ChatBot/PersonalizedBookSearch/goodbooks-10k/to_read.csv', 'r') as file:

            # create a CSV reader object
            reader = csv.reader(file)

            # read the header row
            header = next(reader)

            # create an empty list to store matching rows
            matching_rows = []

            # get the user input
            entered_value = str(user_id) #input('Enter User_id to search:: ')

            # loop through the rows and check for matches
            for row in reader:
                if entered_value == row[0]:

                    with open('/Users/cdmstudent/Desktop/DSC-540 Adv ML/Final Project/ChatBot/PersonalizedBookSearch/goodbooks-10k/books_cleaned.csv', 'r') as file1:
                        reader1 = csv.reader(file1)
                        header = next(reader1)
                        for row1 in reader1:
                            if row[1] == row1[0]:
                                matching_rows.append(row1)

            df = pd.DataFrame(matching_rows, columns = ['Book_id','Author','Title','isbn13'])
            return df

    def direct_input_fav(self,user_id):
    # open the CSV file
        with open('/Users/cdmstudent/Desktop/DSC-540 Adv ML/Final Project/ChatBot/PersonalizedBookSearch/goodbooks-10k/favourite.csv', 'r') as file:

            # create a CSV reader object
            reader = csv.reader(file)

            # read the header row
            header = next(reader)

            # create an empty list to store matching rows
            matching_rows = []

            # get the user input
            entered_value = str(user_id) #input('Enter User_Id to search: ')

            # loop through the rows and check for matches
            for row in reader:
                if entered_value == row[0]:
                    with open('/Users/cdmstudent/Desktop/DSC-540 Adv ML/Final Project/ChatBot/PersonalizedBookSearch/goodbooks-10k/books_cleaned.csv', 'r') as file1:
                        reader1 = csv.reader(file1)
                        header = next(reader1)
                        for row1 in reader1:
                            if row[1] == row1[0]:
                                matching_rows.append(row1)

            df = pd.DataFrame(matching_rows, columns = ['Book_id','Author','Title','isbn13'])
            return df

    def add_to_read(self,book_id,user_id):
        filename = '/Users/cdmstudent/Desktop/DSC-540 Adv ML/Final Project/ChatBot/PersonalizedBookSearch/goodbooks-10k/to_read.csv'

        # Create a new CSV file with headers for two columns User_Id & Book_Id if it doesn't exist
        if not os.path.isfile(filename):
            with open(filename, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['User_Id', 'Book_Id'])  # Write headers for the two columns

        with open(filename, mode='r') as file:
            reader = csv.reader(file)
            i =1
            for row in reader:
                if str(user_id) in row and str(book_id) in row:
                    print("Entered User_Id had this book in the to_read List")
                    i = 0
                    break
                else: continue
        if(i==1):
            # Open the CSV file in append mode
            with open(filename, mode='a', newline='') as file:
                print("Entered User_Id & Book_Id not in your Wishlist List, Now we are going to add into it.") 
                writer = csv.writer(file)
                new_row = [user_id,book_id]

                # Write the new row to the CSV file
                writer.writerow(new_row)
        return

    def add_fav(self,book_id,user_id):
        filename = '/Users/cdmstudent/Desktop/DSC-540 Adv ML/Final Project/ChatBot/PersonalizedBookSearch/goodbooks-10k/favourite.csv'

        # Create a new CSV file with headers for two columns User_Id & Book_Id if it doesn't exist
        if not os.path.isfile(filename):
            with open(filename, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['User_Id', 'Book_Id'])  # Write headers for the two columns

        with open(filename, mode='r') as file:
            reader = csv.reader(file)
            i =1
            for row in reader:
                if str(user_id) in row and str(book_id) in row:
                    print("Entered User_Id had this book in the Favourite List")
                    i = 0
                    break
                else: continue
        if(i==1):
            # Open the CSV file in append mode
            with open(filename, mode='a', newline='') as file:
                print("Entered User_Id & Book_Id not in your Favourite List, Now we are going to add into it.") 
                writer = csv.writer(file)
                new_row = [user_id,book_id]

                # Write the new row to the CSV file
                writer.writerow(new_row)
        return

    def delete_from_fav(self,book_id,user_id):
        filename = '/Users/cdmstudent/Desktop/DSC-540 Adv ML/Final Project/ChatBot/PersonalizedBookSearch/goodbooks-10k/favourite.csv'

        # Read the existing data from the CSV file
        with open(filename, mode='r', newline='') as file:
            reader = csv.reader(file)
            data = list(reader)

        # Find and remove the specified book ID for the given user ID
        updated_data = [row for row in data if row != [user_id, book_id]]

        # Write the updated data back to the CSV file
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(updated_data)
        return

    def delete_from_to_read(self,book_id,user_id):
        filename = '/Users/cdmstudent/Desktop/DSC-540 Adv ML/Final Project/ChatBot/PersonalizedBookSearch/goodbooks-10k/to_read.csv'

        # Read the existing data from the CSV file
        with open(filename, mode='r', newline='') as file:
            reader = csv.reader(file)
            data = list(reader)

        # Find and remove the specified book ID for the given user ID
        updated_data = [row for row in data if row != [user_id, book_id]]

        # Write the updated data back to the CSV file
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(updated_data)
        return

    def transfer_to_fav_from_to_read(self,book_id, user_id):
        filename = '/Users/cdmstudent/Desktop/DSC-540 Adv ML/Final Project/ChatBot/PersonalizedBookSearch/goodbooks-10k/to_read.csv'
        with open(filename, mode='r') as file:
            reader = csv.reader(file)
            #book_id = input("enter book_id: ")
            #user_id = input("enter user_id: ")
            i =1
            for row in reader:
                if user_id in row and book_id in row:
                    print("Entered User_Id had this book in the to_read List")
                    i = 0
                    break
                else: continue
        filename = '/Users/cdmstudent/Desktop/DSC-540 Adv ML/Final Project/ChatBot/PersonalizedBookSearch/goodbooks-10k/favourite.csv'
        # Open the CSV file in append mode
        with open(filename, mode='a', newline='') as file:
            print("Entered User_Id & Book_Id not in your Favourite List, Now we are going to add into it.") 
            writer = csv.writer(file)
            new_row = [user_id,book_id]

            # Write the new row to the CSV file
            writer.writerow(new_row)
        return

    def transfer_to_to_read_from_fav(self,book_id, user_id):
        filename = '/Users/cdmstudent/Desktop/DSC-540 Adv ML/Final Project/ChatBot/PersonalizedBookSearch/goodbooks-10k/favourite.csv'
        with open(filename, mode='r') as file:
            reader = csv.reader(file)
            i =1
            for row in reader:
                if user_id in row and book_id in row:
                    print("Entered User_Id had this book in the favourite List")
                    i = 0
                    break
                else: continue
        filename = '/Users/cdmstudent/Desktop/DSC-540 Adv ML/Final Project/ChatBot/PersonalizedBookSearch/goodbooks-10k/to_read.csv'
        # Open the CSV file in append mode
        with open(filename, mode='a', newline='') as file:
            print("Entered User_Id & Book_Id not in your to_read List, Now we are going to add into it.") 
            writer = csv.writer(file)
            new_row = [user_id,book_id]

            # Write the new row to the CSV file
            writer.writerow(new_row)
        return

    def bot(self,user_id):
        reflections = {
            "i am": "you are",
            "i was": "you were",
            "i": "you",
            "i'm": "you are",
            "i'd": "you would",
            "i've": "you have",
            "i'll": "you will",
            "my": "your",
            "you are": "I am",
            "you were": "I was",
            "you've": "I have",
            "you'll": "I will",
            "your": "my",
            "yours": "mine",
            "you": "me",
            "me": "you"
        }

        pairs = [
            ["my name is (.*)", ["Hello %1, how are you today?"]],
            ["my user_id (.*)",["Ok, your user_id is :%1"]],
            ["myself (.*)", ["Hello %1, how are you today?"]],
            ["hi|hey|hello", ["Hello", "Hey there"]],
            ["(.*) special (.*)|(.*) Speciality|good at what", ["I can find recommendations for you, and i can do multiple tasks one by one but not all at once"]],
            ["(.*) future plans|future plans", ["Mr Bharath Working on it,as of now i'm able to do 60 Percentage multiple tasking soon after i reach 100 this feature will be released untill then single task at a time"]],
            ["what is your name?", ["I'm a chatbot created by Anthropic"]],
            ["how are you(.*)", ["I'm doing good\nHow about You ?"]],
            ["sorry (.*)|sorry", ["Its alright", "Its OK, never mind"]],
            ["I am fine|good|good thanks for asking", ["Great to hear that, How can I help you?"]],
            ["i'm (.*) good|i'm good", ["Nice to hear that, How can I help you?:"]],
            ["(.*) age (.*)", ["I'm a computer program, I don't have an age!"]],
            ["what (.*) want ?|what (.*) want", ["Make me an offer I can't refuse"]],
            ["(.*) created ?", ["Anthropic created me using Python's NLTK library "]],
            ["exit", ["Goodbye!", "See you later!", "Bye!"]]
        ]

        chat = Chat(pairs, reflections)
        user_name = ""

        while True:
            user_input = input("You: ")

            # Check for the exit command
            if any(keyword in user_input for keyword, _ in pairs) and user_input == "exit":
                print("Chatbot: Goodbye!")
                break
            if "my name is" in user_input.lower():
                # Extract the name from the user input
                user_name = user_input.split("my name is")[-1].strip()
                #continue
            elif "what is my name" in user_input.lower():
                # Check if the user has provided their name
                if user_name:
                    print(f"Chatbot: Your name is {user_name}.")
                    continue
                else:
                    print("Chatbot: I don't know your name. Please tell me your name.")
            if "my user_id is" in user_input.lower():
                # Extract the name from the user input
                self.user_id = int(user_input.split("my user_id is")[-1].strip())
                #continue
            elif "what is my user_id" in user_input.lower():
                # Check if the user has provided their name
                if user_id:
                    print(f"Chatbot: Your user_id is {user_id}.")
                    continue
                else:
                    print("Chatbot: I don't know your user_id. Please tell me your user_id.")
            response = chat.respond(user_input)
            print("Chatbot:", response)
            '''if (response == None) or (response == False) :
                try:
                    print("\nuser_id : ", user_id)
                except NoneType:
                    print("\nPlease proide your USer_id\n")
                    continue
                respond_for_input(user_input)'''
            if (user_id > 0):
                self.respond_for_input(user_input)
            else: 
                print("\nPlease proide your USer_id\n")
                continue 

In [61]:
        
if __name__ == "__main__":
    chatbot = Chatbot()
    user_id = int(input("enter your user_id :"))
    #user_inputt = input("What would you like to do? ")
    chatbot.bot(user_id)

enter your user_id :567
You: my name is bharath
Chatbot: Hello bharath, how are you today?
give command

You: find similar books for iron man title
Chatbot: None

combined_word: that is title of the book iron man 

Found recommendations for 'iron man'

    book_id                                            authors  \
0     9779                           Warren Ellis, Adi Granov   
1     1636                                    Patricia Briggs   
2     2267                                       Julie Kagawa   
3     3281                       Holly Black, Cassandra Clare   
4      970                                  Brandon Sanderson   
5     1541                                       Julie Kagawa   
6     8109  Neil Gaiman, Mike Dringenberg, Chris Bachalo, ...   
7     1437                                       Julie Kagawa   
8     4019                             Kurt Busiek, Alex Ross   
9     7883                         Alisa Kwitney, Neil Gaiman   

                              

In [63]:
        
if __name__ == "__main__":
    chatbot = Chatbot()
    user_id = int(input("enter your user_id :"))
    #user_inputt = input("What would you like to do? ")
    chatbot.bot(user_id)

enter your user_id :999
You: what is my user_id
Chatbot: Your user_id is 999.
You: show my to_read list
Chatbot: None

integers in keywords :  []

list of to_read :
   Book_id           Author             Title           isbn13
0    2881  Terry Pratchett      Making Money  9780061161640.0
1    9234       Robin Hobb  Renegade's Magic  9780061473130.0
2       1  Suzanne Collins  The Hunger Games  9780439023480.0
You: find bottom book recommendations from my to_read list
Chatbot: None

integers in keywords :  []

list of to_read in side to find direct recommendations in to_read:
   Book_id           Author             Title           isbn13
0    2881  Terry Pratchett      Making Money  9780061161640.0
1    9234       Robin Hobb  Renegade's Magic  9780061473130.0
2       1  Suzanne Collins  The Hunger Games  9780439023480.0

row:  1
Found recommendations for 'The Hunger Games'

    book_id              authors  \
0        1      Suzanne Collins   
1     6224            Kate Egan   
2      

TypeError: cannot unpack non-iterable NoneType object

In [66]:
# must add to_read list to favorite list vice versa
# unable to fix this
if __name__ == "__main__":
    chatbot = Chatbot()
    user_id = int(input("enter your user_id :"))
    #user_inputt = input("What would you like to do? ")
    chatbot.bot(user_id)

enter your user_id :999
You: transfer book_id 9709 from to_read list into favorite list
Chatbot: None
Copying book 9709 from to_read to favorite
Entered User_Id & Book_Id not in your Favourite List, Now we are going to add into it.
You: show my favorite list
Chatbot: None

integers in keywords :  []

list of favorite :
   Book_id                       Author                       Title  \
0    8841  Neil Gaiman, Michael Reaves                  InterWorld   
1    2881              Terry Pratchett                Making Money   
2    8128           Mary Doria Russell             Children of God   
3    9709          The Harvard Lampoon  The Hunger Pains: A Parody   

            isbn13  
0  9780061238960.0  
1  9780061161640.0  
2  9780449004840.0  
3  9781451668210.0  
You: find similar books for top 3 from favorite llist
Chatbot: None

integers in keywords :  [3]

list of favorite in side to find direct recommendations in fav:
   Book_id                       Author                     